In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys

In [11]:
# importing data
df = pd.read_csv('../2.Downloading data/processed_high_impact_outages.csv', parse_dates=['Start', 'Finish'])

# discuss if this is necessary
bool_cols = ['Project Work?', 'Unplanned?', 'DNSP Aware?', 'Generator Aware?', 'Inter-Regional']
df[bool_cols] = df[bool_cols].applymap(lambda x: True if x == 'T' else False)

df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18722 entries, 0 to 18721
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Region                     18722 non-null  object        
 1   NSP                        18714 non-null  object        
 2   Start                      18504 non-null  datetime64[ns]
 3   Finish                     18504 non-null  datetime64[ns]
 4   Network Asset              18714 non-null  object        
 5   Recall_Day_Hours           18722 non-null  float64       
 6   Recall_Night_Hours         18722 non-null  float64       
 7   Project Work?              18722 non-null  bool          
 8   Unplanned?                 18722 non-null  bool          
 9   DNSP Aware?                18722 non-null  bool          
 10  Generator Aware?           18722 non-null  bool          
 11  Inter-Regional             18722 non-null  bool          
 12  Stat

C:\Users\Jerry Z\AppData\Local\Temp\ipykernel_86784\2433446229.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[bool_cols] = df[bool_cols].applymap(lambda x: True if x == 'T' else False)


,Region,NSP,Start,Finish,Network Asset,Recall_Day_Hours,Recall_Night_Hours,Project Work?,Unplanned?,DNSP Aware?,...,Inter-Regional,Status_Description,Status_Code,Status_Description_Market,Market_Notice_Code,Impact_Region,Reason,Duration_Hours,Impact,file_name
0,NSW,Transgrid,2021-08-30 06:05:00,2021-09-01 12:00:00,Armidale - Dumaresq (8C) 330 kV Line,3.0,4.0,False,False,False,...,False,In Progress,PTP,In Progress,PTP,NSW,Commissioning,52.8,<<UPDATED since the last notification>> A cre...,2021-08-30
1,QLD,Powerlink,2021-09-01 08:30:00,2021-09-01 12:00:00,Ross No.4 288/138/19 kV Transformer,1.0,0.0,False,False,False,...,False,Planned,MTLTP,Planned,MTLTP,QLD,Maintenance,3.5,This is a high impact outage because the recal...,2021-08-30
